In [449]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.window import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json

In [450]:
spark= SparkSession \
       .builder \
       .appName("JsonPyspark") \
       .master("local[*]") \
       .config("spark.dynamicAllocation.enabled", "true") \
       .config("spark.executor.memory", "512mb") \
       .getOrCreate()

In [451]:
df=spark.read.option("mode","DROPMALFORMED").json("datasets/data.json")
df.printSchema()

root
 |-- @context: string (nullable = true)
 |-- @id: string (nullable = true)
 |-- @type: string (nullable = true)
 |-- conformsTo: string (nullable = true)
 |-- dataset: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- @type: string (nullable = true)
 |    |    |-- accessLevel: string (nullable = true)
 |    |    |-- accrualPeriodicity: string (nullable = true)
 |    |    |-- bureauCode: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- contactPoint: struct (nullable = true)
 |    |    |    |-- fn: string (nullable = true)
 |    |    |    |-- hasEmail: string (nullable = true)
 |    |    |-- describedBy: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- distribution: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- @type: string (nullable = true)
 |    |    |    |    |-- accessURL: string (nullable =

In [452]:
updatedDF = df.withColumn("dataset",explode(col("dataset"))).select("dataset.@type","dataset.accesslevel","dataset.accrualPeriodicity","dataset.bureauCode","dataset.contactPoint.fn","dataset.distribution.@type","describedBy")
updatedDF.show()

+------------+-----------+------------------+----------+--------------------+--------------------+--------------------+
|       @type|accesslevel|accrualPeriodicity|bureauCode|                  fn|               @type|         describedBy|
+------------+-----------+------------------+----------+--------------------+--------------------+--------------------+
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...| [dcat:Distribution]|https://project-o...|
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...|[dcat:Distributio...|https://project-o...|
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...|[dcat:Distributio...|https://project-o...|
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...| [dcat:Distribution]|https://project-o...|
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...| [dcat:Distribution]|https://project-o...|
|dcat:Dataset|     public|              

In [453]:
filter_df=updatedDF.filter(updatedDF.accesslevel=="public")


In [454]:
sort_filter=filter_df.sort(filter_df.bureauCode)
sort_filter.show()

+------------+-----------+------------------+----------+--------------------+--------------------+--------------------+
|       @type|accesslevel|accrualPeriodicity|bureauCode|                  fn|               @type|         describedBy|
+------------+-----------+------------------+----------+--------------------+--------------------+--------------------+
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...| [dcat:Distribution]|https://project-o...|
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...|[dcat:Distributio...|https://project-o...|
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...|[dcat:Distributio...|https://project-o...|
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...| [dcat:Distribution]|https://project-o...|
|dcat:Dataset|     public|              null|  [000:00]|Marine Scotland S...| [dcat:Distribution]|https://project-o...|
|dcat:Dataset|     public|              

In [455]:
df_final=df.withColumn("publisher",explode("dataset.publisher"))\
           .withColumn("type",col("publisher.@type"))\
           .withColumn("name",col("publisher.name")) \
           .drop("dataset.publisher","publisher")
display(df_final)

DataFrame[@context: string, @id: string, @type: string, conformsTo: string, dataset: array<struct<@type:string,accessLevel:string,accrualPeriodicity:string,bureauCode:array<string>,contactPoint:struct<fn:string,hasEmail:string>,describedBy:string,description:string,distribution:array<struct<@type:string,accessURL:string,description:string,downloadURL:string,format:string,mediaType:string,title:string>>,identifier:string,issued:string,keyword:array<string>,landingPage:string,modified:string,programCode:array<string>,publisher:struct<@type:string,name:string>,spatial:string,temporal:string,title:string>>, describedBy: string, type: string, name: string]

In [456]:
# from tweepy.models import Status
# import pymongo
# from pymongo import MongoClient
# import tweepy
# import json
# import pandas as pd


# auth = tweepy.OAuthHandler("t0S3HCeFVjL0kG7wsm43rzS3h", "hcxdoNZQTk5sW3dp8HQuZYrL6SFbgtlkzkcdFrBn00k2c2eA4Z")
# auth.set_access_token("2418628328-MwtI3x7wSTKXVvc82VKr3mAouk9JO9BJ3JA35Wf", "hE8OC7cisnM54XcnrSChQeCA6DWZosP85crH8XdQ7Gju0")
# api = tweepy.API(auth,wait_on_rate_limit=True)

# client=MongoClient()
# db=client.tweet_db
# tweet_collection=db.tweet_collection
# tweet_collection.create_index([("id",pymongo.ASCENDING)],unique=True)
# MAX_TWEETS = 50
# df=tweepy.Cursor(api.search_tweets, q='#walmart').items(MAX_TWEETS)
# dff=[]
# for tweet in df:
#     text = tweet._json
# print(text)

In [457]:
# schema= StructType([StructField("eb_id", StringType(),True),
#                    StructField("Zipcode",IntegerType(),True),
#                    StructField("City",StringType(),True),
#                    StructField("State", StringType(),True),
#                    StructField("Location",StringType(),True)])

In [458]:
df_covid=spark.read.option('multiline','true').json('datasets/BP_Optimum_Johor_15092021.json')
df_covid.printSchema()


root
 |-- detail: string (nullable = true)
 |-- eb_id: string (nullable = true)



In [459]:
from pyspark.sql.types import MapType,StringType
Mapschema=MapType(StringType(),StringType())

df1=df_covid.withColumn('detail',from_json(df_covid.detail,Mapschema))
df1.printSchema()


root
 |-- detail: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- eb_id: string (nullable = true)



In [551]:
from pyspark.sql import functions as F

df2=df1.withColumn("parentInformation",df1.detail.parentInformation) \
       .withColumn("listOfHouseholdMember",df1.detail.listOfHouseholdMember) \
       .drop("detail")
df2.show()

# schema=StructType([StructField("parentInformation",StringType(),True)])

# df_final=df_covid.select(from_json(col("detail"),schema)).select("eb_id","detail.*").show()

+----------+--------------------+---------------------+
|     eb_id|   parentInformation|listOfHouseholdMember|
+----------+--------------------+---------------------+
|0101029071|{"A1":"NOH YERENE...| [{"id":"1","flag"...|
|0101029071|{"B6":"PARIT AMAT...|                 null|
|0101029071|{"B6":"PARIT AMAT...|                 null|
|0101029071|{"B6":"PARIT AMAT...|                 null|
|0101029071|{"A1":"KAMARUZAMA...| [{"id":"1","flag"...|
|0101029071|{"B1":"T/L KAL 15...|                 null|
|0101029071|{"B1":"T/L KAL 15...|                 null|
|0101029071|{"A1":"KAMARUDIN ...| [{"id":"1","flag"...|
|0101029071|{"B1":"T/L KAL 15...|                 null|
|0101029071|{"A1":"SALIHAH BI...| [{"id":"1","flag"...|
|0101029071|{"A1":"ASMAH BINT...| [{"id":"1","flag"...|
|0101029071|{"A1":"MADEK BIN ...| [{"id":"1","flag"...|
|0101029071|{"A1":"RAIS BIN A...| [{"id":"1","flag"...|
|0101029071|{"A1":"ISMAIL BIN...| [{"id":"1","flag"...|
|0101029071|{"A1":"KHAMIS BIN...| [{"id":"1","fl

In [552]:
# convertUDF=udf(lambda x:api_call(str(x)))
# schema=StructType([StructField("parentInformation",StringType(),True)])
# df_final=df_covid.withColumn("detail",from_json(convertUDF(col("detail")),schema))
# df_final.show()

In [553]:
import pyspark.sql.functions as F

df3 = df_covid.select(F.get_json_object(F.col("detail"), "$.parentInformation").alias("parentInformation"),
                      F.get_json_object(F.col("detail"), "$.parentInformation.A1").alias("A1"),
                      F.get_json_object(F.col("detail"), "$.parentInformation.B1").alias("B1"),
                      F.get_json_object(F.col("detail"), "$.parentInformation.B6").alias("B6"),
                      F.get_json_object(F.col("detail"), "$.listOfHouseholdMember").alias("listOfHouseholdMember"))
df3.show()

+--------------------+--------------------+----------+--------------------+---------------------+
|   parentInformation|                  A1|        B1|                  B6|listOfHouseholdMember|
+--------------------+--------------------+----------+--------------------+---------------------+
|{"A1":"NOH YERENE...|NOH YERENEES BIN ...|      null|   PARIT AMAT RENGIT| [{"id":"1","flag"...|
|{"B6":"PARIT AMAT...|                null|      null|   PARIT AMAT RENGIT|                 null|
|{"B6":"PARIT AMAT...|                null|      null|   PARIT AMAT RENGIT|                 null|
|{"B6":"PARIT AMAT...|                null|      null|   PARIT AMAT RENGIT|                 null|
|{"A1":"KAMARUZAMA...|KAMARUZAMAN BIN K...|      null|KAMPUNG PARIT AMA...| [{"id":"1","flag"...|
|{"B1":"T/L KAL 15...|                null|T/L KAL 15|   PARIT AMAT RENGIT|                 null|
|{"B1":"T/L KAL 15...|                null|T/L KAL 15|   PARIT AMAT RENGIT|                 null|
|{"A1":"KAMARUDIN ..

In [554]:
# from pyspark.sql.types import *
# import pyspark.sql.functions as F

# aSchema = StructType([
#     StructField("c", StructType([
#         StructField("60", DoubleType(), True)
#     ]), True),
#     StructField("n", StructType([
#         StructField("60", DoubleType(), True)
#     ]), True),
#     StructField("d", DoubleType(), True),
# ])
# gSchema = StructType([
#     StructField("h", DoubleType(), True)
# ])

# schema = StructType([
#     StructField("a", aSchema, True),
#     StructField("g", gSchema, True)
# ])

# df = spark.createDataFrame(['{"a":{"b":0,"c":{"50":0.005,"60":0,"100":0},"d":0.01,"e":0,"f":2}}',
#                             '{"a":{"m":0,"n":{"50":0.005,"60":0,"100":0},"d":0.01,"e":0,"f":2}}',
#                             '{"g":{"h":0,"j":{"50":0.005,"80":0,"100":0},"d":0.02}}'],
#                            StringType())

# df2 = df.select(F.from_json("value", schema=schema).alias('data')).select('data.*')
# df2.select(df2.a.c['60'], df2.a.n['60'], df2.a.d, df2.g.h).show()

In [555]:
# df_details =  df_covid.select(col("eb_id"), \
#      from_json(col("detail"),schema).alias("detail")) \
#        .select("eb_id","detail.*")
# df_details.show()

In [556]:

# import pyspark.sql.functions as F
 
# parsed_df = df_covid.select(F.col('eb_id'), F.col("detail.parentInformation"))
# parsed_df.show(truncate=False)
# parsed_df.printSchema()

In [557]:
df_location=df_covid.select(df_covid.detail.[0])
df_location.show()

SyntaxError: invalid syntax (2294310390.py, line 1)

In [501]:
#aa=df_covid.select(col('detail')).rdd.map(lambda x: json.loads(x.detail))

In [502]:
df_null=df_covid.filter((df_covid.Location.isNull()) & (df_covid.eb_id.isNotNull()))
df_null.show()

AttributeError: 'DataFrame' object has no attribute 'Location'